In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'eli5'

In [10]:
data= pd.read_excel("case_study.xlsx")

In [11]:
data.shape

(150, 4)

In [12]:
data['year']= data['ISO_Week'].apply(lambda x:x.split('-')[0])
data['week_no']= data['ISO_Week'].apply(lambda x:x.split('-')[1])
data['week_no']= data['week_no'].apply(lambda x:int((x)))

In [13]:
data

,SKU,ISO_Week,Sales,Season,year,week_no
0,ProductA,2018-01,0.0,WINTER,2018,1
1,ProductA,2018-02,0.0,WINTER,2018,2
2,ProductA,2018-03,0.0,WINTER,2018,3
3,ProductA,2018-04,6988.0,WINTER,2018,4
4,ProductA,2018-04,6988.0,WINTER,2018,4
...,...,...,...,...,...,...
145,ProductC,2018-48,12927.0,AUTUMN,2018,48
146,ProductC,2018-49,11595.0,WINTER,2018,49
147,ProductC,2018-50,10061.0,WINTER,2018,50
148,ProductC,2018-51,10859.0,WINTER,2018,51


In [14]:
train = data.loc[(data['week_no'] >= 1) & (data['week_no'] <= 41)]
test = data.loc[(data['week_no'] >= 42) & (data['week_no'] <= 52)]
test= test.reset_index(drop=True)

In [15]:
train

,SKU,ISO_Week,Sales,Season,year,week_no
0,ProductA,2018-01,0.0,WINTER,2018,1
1,ProductA,2018-02,0.0,WINTER,2018,2
2,ProductA,2018-03,0.0,WINTER,2018,3
3,ProductA,2018-04,6988.0,WINTER,2018,4
4,ProductA,2018-04,6988.0,WINTER,2018,4
...,...,...,...,...,...,...
134,ProductC,2018-37,11856.0,AUTUMN,2018,37
135,ProductC,2018-38,4185.0,AUTUMN,2018,38
136,ProductC,2018-39,4357.0,AUTUMN,2018,39
137,ProductC,2018-40,3730.0,AUTUMN,2018,40


In [16]:
#Preprocessing
train= train.loc[(train['Sales'] != 0.0)]
train= train.reset_index(drop=True)
print(train.shape)
train.drop_duplicates(keep=False,inplace=True)
print(train.shape)
train=train.reset_index(drop=True)

(100, 6)
(92, 6)


In [17]:
train.head(5)

,SKU,ISO_Week,Sales,Season,year,week_no
0,ProductA,2018-05,6743.0,WINTER,2018,5
1,ProductA,2018-06,4112.0,WINTER,2018,6
2,ProductA,2018-07,5732.0,WINTER,2018,7
3,ProductA,2018-08,NaN,WINTER,2018,8
4,ProductA,2018-09,5559.0,SPRING,2018,9


In [18]:
train['Sales'].fillna(train.groupby(['SKU'])['Sales'].transform('mean'),inplace=True)
train.head(5)

,SKU,ISO_Week,Sales,Season,year,week_no
0,ProductA,2018-05,6743.00000,WINTER,2018,5
1,ProductA,2018-06,4112.00000,WINTER,2018,6
2,ProductA,2018-07,5732.00000,WINTER,2018,7
3,ProductA,2018-08,6898.84375,WINTER,2018,8
4,ProductA,2018-09,5559.00000,SPRING,2018,9


In [20]:
# le = LabelEncoder()
# train['Season']=  train['Season'].astype('str')
# train['Season']= le.fit_transform(train['Season'])
#Frequency Encoding
combine_set=pd.concat([train,test])
combine_set['SKU']=combine_set['SKU'].map({'ProductA':1,'ProductB':2,'ProductC':3})
#Frequency Encoding
fe=combine_set.groupby(['Season']).size()/len(combine_set)
combine_set['Season']=combine_set['Season'].apply(lambda x: fe[x])
combine_set['Number_of_times_product_sold']=combine_set.groupby(['SKU','Season'])['Season'].transform('count')
combine_set['Average_times_product_sold_during_season']=combine_set.groupby(['SKU','Season'])['Season'].transform('mean')


In [21]:
combine_set['Sales'].max()

52524.0

In [24]:
np.log1p([6743])

array([8.8164085])

In [25]:
combine_set['New_Sales']=np.log1p(combine_set['Sales']+164)
combine_set['New_Sales_2']=np.log1p(combine_set['Sales'])

In [26]:
combine_set[['Sales','New_Sales','New_Sales_2']]

,Sales,New_Sales,New_Sales_2
0,6743.00000,8.840435,8.816408
1,4112.00000,8.361007,8.321908
2,5732.00000,8.682199,8.653994
3,6898.84375,8.862745,8.839254
4,5559.00000,8.652423,8.623353
...,...,...,...
28,12927.00000,9.479757,9.467151
29,11595.00000,9.372459,9.358415
30,10061.00000,9.232689,9.216521
31,10859.00000,9.307830,9.292842


In [ ]:
combine_set['year']=combine_set['year'].astype('int64')
train=combine_set[combine_set['week_no']<42]
test=combine_set[combine_set['week_no']>=42]
X=train.drop(['ISO_Week','Sales'],axis=1)
y=train['Sales']

X_main_test=test.drop(['ISO_Week','Sales'],axis=1)
y_val=test['Sales']

In [ ]:
combine_set.head(5)

In [ ]:
m = LGBMRegressor(boosting_type='gbdt',n_estimators=1000, 
                  num_leaves=3, 
                  max_depth=4,
                  min_child_samples=2,
                  learning_rate=0.04,
                  colsample_bytree=0.8,
                  reg_alpha=2,
                  reg_lambda=2)
m.fit(X, y)

In [ ]:
pred = m.predict(X_main_test)
print(np.sqrt(mean_squared_error(y_val,pred)))


In [ ]:
#Weight Correction for better RMSE
print("weight correction")
W=[(0.990+(i/1000)) for i in range(100)]
S =[]
for w in W:
    error = np.sqrt(mean_squared_error(y_val, pred*w))
    print('RMSE for {:.3f}:{:.6f}'.format(w,error))
    S.append(error)
Score = pd.DataFrame(S,index=W)

BS = Score[Score.values == Score.values.min()]
print ('Best weight for Score:{}'.format(BS))

In [ ]:
#Final file 
d=pd.DataFrame()
d['SKU']=test['SKU'].map({1:'ProductA',2:'ProductB',3:'ProductC'})
d['ISO_Week']=test['ISO_Week']
d['Forecast']=pd.DataFrame(np.expm1(pred*BS.index.values)-164)
d.to_excel('C:/Users/Tahreem Haque/Downloads/main_test.xlsx',index=False)
d.head(5)

In [ ]:
# tra_a= train_final_2[train_final_2['SKU']=='ProductA'].reset_index(drop=True)

In [ ]:
# df= tra_a[['Sales','week_no']]

In [ ]:
# ax = df.plot(figsize=(14,6), lw=2, style='k-')


In [ ]:
# import matplotlib.pyplot as plt


In [ ]:
# plt.hist(train_final_2['Sales'], bins = 20, color = 'pink')
# plt.title('Target Variable')
# plt.xlabel('ItemSales')
# plt.ylabel('count')
# plt.show()

In [ ]:
# x = range(len(train_final_2['Sales']))
# plt.plot(x, train_final_2['Sales'])
# plt.xticks(x, zip(train_final_2['year'], train_final_2['week_no']))

In [ ]:
pred